<img src="../../../images/banners/pandas-cropped.jpeg" width="600"/>

<a class="anchor" id="intro_to_data_structures"></a>
# <img src="../../../images/logos/pandas.png" width="23"/> DataFrame

## <img src="../../../images/logos/toc.png" width="20"/> Table of Contents
* [Create a Dataframe](#create-dataframe)
    * [From dict of Series or dicts](#from_dict_of_series_or_dicts)
    * [From dict of ndarrays / lists](#from_dict_of_ndarrays_/_lists)
    * [From structured or record array](#from_structured_or_record_array)
    * [From a list of dicts](#from_a_list_of_dicts)
    * [From a dict of tuples](#from_a_dict_of_tuples)
    * [From a Series](#from_a_series)
    * [From a list of namedtuples](#from_a_list_of_namedtuples)
    * [From a list of dataclasses](#from_a_list_of_dataclasses)
    * [Alternate constructors](#alternate_constructors)
* [Column selection, addition, deletion](#column_selection,_addition,_deletion)

---

In [2]:
import pandas as pd
import numpy as np

**DataFrame** is a 2-dimensional labeled data structure with columns of
potentially different types. You can think of it like a spreadsheet or SQL
table, or a dict of Series objects. It is generally the most commonly used
pandas object.

<a class="anchor" id="create-dataframe"></a>
## Create a DataFrame

Like Series, DataFrame accepts many different kinds of input:
- Dict of 1D ndarrays, lists, dicts, or Series
- 2-D numpy.ndarray
- Structured or record ndarray
- A `Series`
- Another `DataFrame`

Along with the data, you can optionally pass **index** (row labels) and
**columns** (column labels) arguments. If you pass an index and / or columns,
you are guaranteeing the index and / or columns of the resulting
DataFrame. Thus, a dict of Series plus a specific index will discard all data
not matching up to the passed index.

If axis labels are not passed, they will be constructed from the input data
based on common sense rules.

> **Note:**
> When the data is a dict, and `columns` is not specified, the `DataFrame`
columns will be ordered by the dict’s insertion order, if you are using
Python version >= 3.6 and pandas >= 0.23.
> 
> If you are using Python < 3.6 or pandas < 0.23, and `columns` is not
specified, the `DataFrame` columns will be the lexically ordered list of dict
keys.

<a class="anchor" id="from_dict_of_series_or_dicts"></a>
### From dict of Series or dicts

The resulting **index** will be the **union** of the indexes of the various
Series. If there are any nested dicts, these will first be converted to
Series. If no columns are passed, the columns will be the ordered list of dict
keys.

In [3]:
d = {
    "one": pd.Series([1.0, 2.0, 3.0], index=["a", "b", "c"]),
    "two": pd.Series([1.0, 2.0, 3.0, 4.0], index=["a", "b", "c", "d"]),
}

In [4]:
df = pd.DataFrame(d)

In [5]:
df

,one,two
a,1.0,1.0
b,2.0,2.0
c,3.0,3.0
d,NaN,4.0


In [6]:
pd.DataFrame(d, index=["d", "b", "a"])

,one,two
d,NaN,4.0
b,2.0,2.0
a,1.0,1.0


In [7]:
pd.DataFrame(d, index=["d", "b", "a"], columns=["two", "three"])

,two,three
d,4.0,NaN
b,2.0,NaN
a,1.0,NaN


The row and column labels can be accessed respectively by accessing the
**index** and **columns** attributes:

> **Note:**
> When a particular set of columns is passed along with a dict of data, the
passed columns override the keys in the dict.

In [8]:
df.index

Index(['a', 'b', 'c', 'd'], dtype='object')

In [9]:
df.columns

Index(['one', 'two'], dtype='object')

<a class="anchor" id="from_dict_of_ndarrays_/_lists"></a>
### From dict of ndarrays / lists

The ndarrays must all be the same length. If an index is passed, it must
clearly also be the same length as the arrays. If no index is passed, the
result will be `range(n)`, where `n` is the array length.

In [10]:
d = {"one": [1.0, 2.0, 3.0, 4.0], "two": [4.0, 3.0, 2.0, 1.0]}

In [11]:
pd.DataFrame(d)

,one,two
0,1.0,4.0
1,2.0,3.0
2,3.0,2.0
3,4.0,1.0


In [12]:
pd.DataFrame(d, index=["a", "b", "c", "d"])

,one,two
a,1.0,4.0
b,2.0,3.0
c,3.0,2.0
d,4.0,1.0


<a class="anchor" id="from_structured_or_record_array"></a>
### From structured or record array

This case is handled identically to a dict of arrays.

In [13]:
data = np.zeros((2,), dtype=[("A", "i4"), ("B", "f4"), ("C", "a10")])

In [14]:
data[:] = [(1, 2.0, "Hello"), (2, 3.0, "World")]

In [15]:
pd.DataFrame(data)

,A,B,C
0,1,2.0,b'Hello'
1,2,3.0,b'World'


In [16]:
pd.DataFrame(data, index=["first", "second"])

,A,B,C
first,1,2.0,b'Hello'
second,2,3.0,b'World'


In [17]:
pd.DataFrame(data, columns=["C", "A", "B"])

,C,A,B
0,b'Hello',1,2.0
1,b'World',2,3.0


> **Note:**
> DataFrame is not intended to work exactly like a 2-dimensional NumPy
ndarray.

<a class="anchor" id="from_a_list_of_dicts"></a>
### From a list of dicts

In [18]:
data2 = [{"a": 1, "b": 2}, {"a": 5, "b": 10, "c": 20}]

In [19]:
pd.DataFrame(data2)

,a,b,c
0,1,2,NaN
1,5,10,20.0


In [20]:
pd.DataFrame(data2, index=["first", "second"])

,a,b,c
first,1,2,NaN
second,5,10,20.0


In [21]:
pd.DataFrame(data2, columns=["a", "b"])

,a,b
0,1,2
1,5,10


<a class="anchor" id="from_a_dict_of_tuples"></a>
### From a dict of tuples

You can automatically create a MultiIndexed frame by passing a tuples
dictionary.

In [22]:
pd.DataFrame(
    {
        ("a", "b"): {("A", "B"): 1, ("A", "C"): 2},
        ("a", "a"): {("A", "C"): 3, ("A", "B"): 4},
        ("a", "c"): {("A", "B"): 5, ("A", "C"): 6},
        ("b", "a"): {("A", "C"): 7, ("A", "B"): 8},
        ("b", "b"): {("A", "D"): 9, ("A", "B"): 10},
    }
)


a              b      
       b    a    c    a     b
A B  1.0  4.0  5.0  8.0  10.0
  C  2.0  3.0  6.0  7.0   NaN
  D  NaN  NaN  NaN  NaN   9.0

<a class="anchor" id="from_a_series"></a>
### From a Series

The result will be a DataFrame with the same index as the input Series, and
with one column whose name is the original name of the Series (only if no other
column name provided).

<a class="anchor" id="from_a_list_of_namedtuples"></a>
### From a list of namedtuples

The field names of the first `namedtuple` in the list determine the columns
of the `DataFrame`. The remaining namedtuples (or tuples) are simply unpacked
and their values are fed into the rows of the `DataFrame`. If any of those
tuples is shorter than the first `namedtuple` then the later columns in the
corresponding row are marked as missing values. If any are longer than the
first `namedtuple`, a `ValueError` is raised.

In [23]:
from collections import namedtuple

In [24]:
Point = namedtuple("Point", "x y")

In [25]:
pd.DataFrame([Point(0, 0), Point(0, 3), (2, 3)])

,x,y
0,0,0
1,0,3
2,2,3


In [26]:
Point3D = namedtuple("Point3D", "x y z")

In [27]:
pd.DataFrame([Point3D(0, 0, 0), Point3D(0, 3, 5), Point(2, 3)])

,x,y,z
0,0,0,0.0
1,0,3,5.0
2,2,3,NaN


<a class="anchor" id="from_a_list_of_dataclasses"></a>
### From a list of dataclasses

New in version 1.1.0.

Data Classes as introduced in [PEP557](https://www.python.org/dev/peps/pep-0557),
can be passed into the DataFrame constructor.
Passing a list of dataclasses is equivalent to passing a list of dictionaries.

Please be aware, that all values in the list should be dataclasses, mixing
types in the list would result in a TypeError.

In [28]:
from dataclasses import make_dataclass
Point = make_dataclass("Point", [("x", int), ("y", int)])
pd.DataFrame([Point(0, 0), Point(0, 3), Point(2, 3)])

,x,y
0,0,0
1,0,3
2,2,3


<a class="anchor" id="alternate_constructors"></a>
### Alternate constructors

**DataFrame.from_dict**

`DataFrame.from_dict` takes a dict of dicts or a dict of array-like sequences
and returns a DataFrame. It operates like the `DataFrame` constructor except
for the `orient` parameter which is `'columns'` by default, but which can be
set to `'index'` in order to use the dict keys as row labels.

In [29]:
pd.DataFrame.from_dict(dict([("A", [1, 2, 3]), ("B", [4, 5, 6])]))

,A,B
0,1,4
1,2,5
2,3,6


If you pass `orient='index'`, the keys will be the row labels. In this
case, you can also pass the desired column names:

In [30]:
pd.DataFrame.from_dict(
    dict([("A", [1, 2, 3]), ("B", [4, 5, 6])]),
    orient="index",
    columns=["one", "two", "three"],
)


,one,two,three
A,1,2,3
B,4,5,6


**DataFrame.from_records**

`DataFrame.from_records` takes a list of tuples or an ndarray with structured
dtype. It works analogously to the normal `DataFrame` constructor, except that
the resulting DataFrame index may be a specific field of the structured
dtype. For example:

In [31]:
data

array([(1, 2., b'Hello'), (2, 3., b'World')],
      dtype=[('A', '<i4'), ('B', '<f4'), ('C', 'S10')])

In [32]:
pd.DataFrame.from_records(data, index="C")

,A,B
C,,
b'Hello',1,2.0
b'World',2,3.0


<a class="anchor" id="column_selection,_addition,_deletion"></a>
## Column selection, addition, deletion

You can treat a DataFrame semantically like a dict of like-indexed Series
objects. Getting, setting, and deleting columns works with the same syntax as
the analogous dict operations:

In [33]:
df["one"]

a    1.0
b    2.0
c    3.0
d    NaN
Name: one, dtype: float64

In [34]:
df["three"] = df["one"] * df["two"]

In [35]:
df["flag"] = df["one"] > 2

In [36]:
df

,one,two,three,flag
a,1.0,1.0,1.0,False
b,2.0,2.0,4.0,False
c,3.0,3.0,9.0,True
d,NaN,4.0,NaN,False


Columns can be deleted or popped like with a dict:

In [37]:
del df["two"]

In [38]:
three = df.pop("three")

In [39]:
df

,one,flag
a,1.0,False
b,2.0,False
c,3.0,True
d,NaN,False


When inserting a scalar value, it will naturally be propagated to fill the
column:

In [40]:
df["foo"] = "bar"

In [41]:
df

,one,flag,foo
a,1.0,False,bar
b,2.0,False,bar
c,3.0,True,bar
d,NaN,False,bar


When inserting a Series that does not have the same index as the DataFrame, it
will be conformed to the DataFrame’s index:

In [42]:
df["one_trunc"] = df["one"][:2]

In [43]:
df

,one,flag,foo,one_trunc
a,1.0,False,bar,1.0
b,2.0,False,bar,2.0
c,3.0,True,bar,NaN
d,NaN,False,bar,NaN


You can insert raw ndarrays but their length must match the length of the
DataFrame’s index.

By default, columns get inserted at the end. The `insert` function is
available to insert at a particular location in the columns:

In [44]:
df.insert(1, "bar", df["one"])

In [45]:
df

,one,bar,flag,foo,one_trunc
a,1.0,1.0,False,bar,1.0
b,2.0,2.0,False,bar,2.0
c,3.0,3.0,True,bar,NaN
d,NaN,NaN,False,bar,NaN
